In [1]:
%%capture
import numpy as np
import pandas as pd
from healpy.sphtfunc import smoothing
import smica
import os
import json

import copy
from astropy.io import fits
from scipy import interpolate

import healpy as hp
import component_separation
from component_separation.io import IO
import component_separation.interface as cslib
import component_separation.MSC.MSC.pospace as ps
import component_separation.powspec as pw

from component_separation.cs_util import Config
from component_separation.cs_util import Constants as const
from component_separation.cs_util import Helperfunctions as hpf
from component_separation.cs_util import Planckf, Planckr, Plancks
import matplotlib.pyplot as plt
csu = Config()
io = IO(csu)
import warnings
warnings.filterwarnings('ignore')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

lmax=csu.lmax
nside_out = csu.nside_out
bins = csu.bins
detector = csu.PLANCKMAPFREQ_f
specfilter = csu.specfilter
freqfilter = csu.freqfilter
lmaxbins = int(bins[-1][1])
colors=csu.CB_color_cycle
filename = io.fh.total_filename
bcorr = [
    hp.gauss_beam(val, lmax, pol = True)[:,1]
    for val in [
        0.00930842,
        0.00785398,
        0.00378155,
        0.002807071,
        0.002106031,
        0.00145444,
        0.00140499,
        ]
    ]
beam5 = hp.gauss_beam(0.00145444, lmax, pol = True)[:,1]

In [6]:
# NP_smicaclmin = io.load_data(io.fh.clmin_smica_path_name)

cfc = copy.deepcopy(csu.cf)
cfc['pa']['freqdset'] = 'NPIPE'
cfc['pa']['mskset'] = 'lens'
csuc = Config(cfc)
ioc = IO(csuc) 
NP_lensclmin = ioc.load_data(ioc.fh.clmin_smica_path_name)
cfc = copy.deepcopy(csu.cf)
cfc['pa']['freqdset'] = 'DX12'
cfc['pa']['mskset'] = 'smica'
csuc = Config(cfc)
ioc = IO(csuc) 
DX_smicaclmin = ioc.load_data(ioc.fh.clmin_smica_path_name)
cfc = copy.deepcopy(csu.cf)
cfc['pa']['freqdset'] = 'DX12'
cfc['pa']['mskset'] = 'lens'
csuc = Config(cfc)
ioc = IO(csuc) 
DX_lensclmin = ioc.load_data(ioc.fh.clmin_smica_path_name)

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Run with the following settings:
{'Tscale': 'K_CMB', 'mskset': 'lens', 'freqdset': 'NPIPE', 'Spectrum_scale': 'C_l', 'lmax': 4000, 'lmax_mask': 6000, 'freqdatsplit': '', 'smoothing_window': 0, 'max_polynom': 0, 'num_sim': 5, 'binname': 'SMICA_highell_bins', 'overwrite_cache': True, 'split': '', 'nside_out': None, 'splitvariation': 'GHz', 'freqfilter': ['545', '857'], 'specfilter': ['TB', 'EB', 'ET', 'BT', 'BE'], 'nside_desc_map': [1024, 2048]}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
loaded /global/cscratch1/sd/sebibel/smica/smicaclmin_SMICA_highell_bins_0200_C_l_NPIPE_lens_4000_6000_0_0_Full.npy
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Run with the following settings:
{'Tscale': 'K_CMB', 'mskset': 'smica', 'freqdset': 'DX12', 'Spectrum_scale': 'C_l', 'lmax': 4000, 'lmax_mask': 6000, 'freqdatsplit': '', 'smoothing_window': 0, 'max_polynom': 0, 'num_sim': 5, 'binname': 'SMICA_highell_bins', 'overwrite_cache': True, 'split': '', 'nside_out': None,

In [10]:
%matplotlib widget
# print(NP_smicaclmin.shape)
# print(DX_smicaclmin.shape)
# plt.plot(NP_smicaclmin[1], label='NPIPE smica')
plt.plot(NP_lensclmin[1], label='NPIPE lens')
# plt.plot(smica_C_lmin_sc[1], label='NPIPE data with DX12-smica weights', marker='.')
# plt.plot(propag_smica_C_lmin_sc[1], label='NPIPE data with smica-propag weights', marker='.')
# plt.plot(DX_smicaclmin[0], label='DX12')
plt.legend()
# plt.plot(C_lS[0,1,:])
plt.yscale('log')
plt.ylabel('C_l')
plt.title('BB')
plt.xlabel('Multipole')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Multipole')

## Variance

In [ ]:
EBidx = 1
ll = np.arange(0,4000,1)
approx_variance_NPDXsmica = 2 * smica_C_lmin_sc[EBidx,:] * smica_C_lmin_sc[EBidx,:]/((2*ll+1)*0.67)
approx_variance_NPlens = 2 * NP_lensclmin[EBidx,:] * NP_lensclmin[EBidx,:]/((2*ll+1)*0.67)
approx_variance_NPpropag = 2 * propag_smica_C_lmin_sc[EBidx,:] * propag_smica_C_lmin_sc[EBidx,:]/((2*ll+1)*0.67)

In [ ]:
%matplotlib widget
# plt.plot(approx_variance_NPDXsmica)
# plt.plot(approx_variance_NPlens)

mean, std, _ = std_dev_binned(approx_variance_NPlens/approx_variance_NPDXsmica-1, lmax=4000, binwidth=50, log=False)
plt.errorbar((_[1:]+_[:-1])/2, mean, yerr=std, fmt='.', capsize=5, label= 'NPIPE data with NPIPE-lens weights\nover\nNPIPE data with DX12-smica weights')
mean, std, _ = std_dev_binned(approx_variance_NPlens/approx_variance_NPpropag-1, lmax=4000, binwidth=50, log=False)
plt.errorbar((_[1:]+_[:-1])/2, mean, yerr=std, fmt='.', capsize=5, label= 'NPIPE data with NPIPE-lens weights\nover\nNPIPE data with smica-propag weights')


# mean, std, _ = std_dev_binned(NP_lensclmin[EBidx,:]/smica_C_lmin_sc[EBidx,:]-1, lmax=4000, binwidth=50, log=False)
# plt.errorbar((_[1:]+_[:-1])/2, mean, yerr=std, fmt='.', capsize=5, label= 'NPIPE data with NPIPE-lens weights\nover\nNPIPE data with DX12-smica weights')
# mean, std, _ = std_dev_binned(NP_lensclmin[EBidx,:]/propag_smica_C_lmin_sc[EBidx,:]-1, lmax=4000, binwidth=50, log=False)
# plt.errorbar((_[1:]+_[:-1])/2, mean, yerr=std, fmt='.', capsize=5, label= 'NPIPE data with NPIPE-lens weights\nover\nNPIPE data with smica-propag weights')


plt.hlines(0,lmax,0, ls='--', color= 'gray')
plt.ylim((-0.9,0.1))
# plt.xscale('log')
plt.xlim((1e1,4e3))
plt.legend()
plt.xlabel('Multipole')
plt.ylabel('Rel. Diff.')
plt.title('Comparing Variance of MV spectra')
# plt.yscale('log')

In [68]:
%matplotlib widget
# mean, std, _ = std_dev_binned(1-(NP_lensclmin[0]/NP_smicaclmin[0]), lmax=4000, binwidth=25, log=False)
# plt.errorbar((_[1:]+_[:-1])/2, mean, yerr=std, fmt='.', capsize=5, label='NP_lens/NP_smica')
mean, std, _ = std_dev_binned(1-(NP_lensclmin[0]/smica_C_lmin_sc[0]), lmax=4000, binwidth=25, log=False)
plt.errorbar((_[1:]+_[:-1])/2, mean, yerr=std, fmt='.', capsize=5, label='NP_lens/NP_DX_smica')
mean, std, _ = std_dev_binned(1-(NP_lensclmin[0]/DX_smicaclmin[0]), lmax=4000, binwidth=25, log=False)
plt.errorbar((_[1:]+_[:-1])/2, mean, yerr=std, fmt='.', capsize=4, label='NP_lens/DX_smica', alpha=0.5)
mean, std, _ = std_dev_binned(1-(NP_smicaclmin[0]/DX_smicaclmin[0]), lmax=4000, binwidth=25, log=False)
plt.errorbar((_[1:]+_[:-1])/2, mean, yerr=std, fmt='.', capsize=4, label='NP_smica/DX_smica', alpha=0.5)
plt.legend()
plt.hlines(0,lmax,0, ls='--', color= 'gray')
plt.ylim((-0.2,0.5))

(4000,) (25,) (4000,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4000,) (25,) (4000,)
(4000,) (25,) (4000,)


(-0.2, 0.5)

## Take DX12-smica weights, apply on NPIPE data
 * 1. via public weights
 * 2. via self derived

In [69]:
## with self-derived weights

cfc = copy.deepcopy(csu.cf)
cfc['pa']['freqdset'] = 'DX12'
cfc['pa']['mskset'] = 'smica'
csuc = Config(cfc)
ioc = IO(csuc) 
"""
Follows the SMICA propagation code to combine maps with set of weights.
Only runs for the chosen bins and up to the max value of bins
"""
CMB = dict()
CMB["TQU"] = dict()
almT, almE, almB = dict(), dict(), dict()
lmaxbin = int(bins[-1][1]+1)
lmax = csu.lmax

W_smica = ioc.load_data(ioc.fh.weight_smica_path_name)
W_mv = ioc.load_data(ioc.fh.weight_path_name)
W_total = np.zeros(shape=(*W_mv.shape[:-1], csu.lmax+1))

xnew = np.arange(0,lmaxbin+1,1)
for it, det in enumerate(csu.PLANCKMAPFREQ): #weights do not depend on freqfilter, but almE/B do
    if det in csu.PLANCKMAPFREQ_f:
        ns = csu.nside_out[0] if int(det) < 100 else csu.nside_out[1]
        W_Einterp = interpolate.interp1d(np.mean(bins, axis=1), W_smica[0,it,:], bounds_error = False, fill_value='extrapolate')
        W_total[1,it] = np.concatenate((W_Einterp(xnew),W_mv[1,it,xnew.shape[0]:]))
        W_Binterp = interpolate.interp1d(np.mean(bins, axis=1), W_smica[1,it,:], bounds_error = False, fill_value='extrapolate')
        W_total[2,it] = np.concatenate((W_Binterp(xnew),W_mv[2,it,xnew.shape[0]:]))

cfc = copy.deepcopy(csu.cf)
cfc['pa']['freqdset'] = 'NPIPE'
cfc['pa']['mskset'] = 'lens'
csuc = Config(cfc)
ioc = IO(csuc) 
# full maps
maps = ioc.load_plamap(csuc.cf, field=(0,1,2), nside_out=csu.nside_out)
# maps = trsf_m.process_all(maps)
beamf = ioc.load_beamf(freqcomb=csu.freqcomb)
tmask, pmask, pmask =  io.load_one_mask_forallfreq()

for freq in csu.PLANCKMAPFREQ_f:
    print('freq: ', freq)
    ns = csu.nside_out[0] if int(freq) < 100 else csu.nside_out[1]
    alms = pw.map2alm_spin(maps[freq], hp.ud_grade(pmask[freq], nside_out=ns), 2, lmax-1) # full sky QU->EB
    # almT[det] = alms[0]
    almE[freq] = alms[0]
    almB[freq] = alms[1]

nalm = int((lmax)*(lmax-1+2)/2)  
# combalmT = np.zeros((nalm), dtype=np.complex128)
combalmE = np.zeros((nalm), dtype=np.complex128)
combalmB = np.zeros((nalm), dtype=np.complex128)


for it, det in enumerate(csu.PLANCKMAPFREQ): #weights do not depend on freqfilter, but almE/B do
    if det in csu.PLANCKMAPFREQ_f:
        print('freq: ', det)
        ns = csu.nside_out[0] if int(det) < 100 else csu.nside_out[1]
        # combalmT += hp.almxfl(almT[name], np.squeeze(W[0,m,:]))
        LHFI = "LFI" if int(det)<100 else "HFI"
        if csu.cf['pa']['freqdset'].startswith('NPIPE'):
            LHFI = "HFI"
        combalmE += hp.almxfl(hp.almxfl(almE[det],1/beamf[str(det)+'-'+str(det)][LHFI][1].data.field(1)[:lmax]), np.squeeze(W_total[1,it,:]))
        combalmE = hp.almxfl(combalmE, 1/hp.pixwin(ns, pol=True)[0][:lmax])
#             combalmE = hp.smoothalm(combalmE, fwhm = np.radians(5/60))
        combalmB += hp.almxfl(hp.almxfl(almB[det],1/beamf[str(det)+'-'+str(det)][LHFI][1].data.field(2)[:lmax]), np.squeeze(W_total[2,it,:]))
        combalmB = hp.almxfl(combalmB, 1/hp.pixwin(ns, pol=True)[1][:lmax])
#             combalmB = hp.smoothalm(combalmB, fwhm = np.radians(5/60))

CMB["TQU"]['out'] = hp.alm2map([np.zeros_like(combalmE), combalmE, combalmB], csu.nside_out[1])
smica_C_lmin_sc = np.array(ps.map2cl_spin(qumap=CMB["TQU"]['out'][1:3], spin=2, mask=pmask['100'], lmax=lmax-1,
    lmax_mask=lmax*2))*1e12 #maps are different scale than processed powerspectra from this' package pipeline, thus *1e12

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Run with the following settings:
{'Tscale': 'K_CMB', 'mskset': 'smica', 'freqdset': 'DX12', 'Spectrum_scale': 'C_l', 'lmax': 4000, 'lmax_mask': 6000, 'freqdatsplit': '', 'smoothing_window': 0, 'max_polynom': 0, 'num_sim': 5, 'binname': 'SMICA_highell_bins', 'overwrite_cache': True, 'split': '', 'nside_out': None, 'splitvariation': 'GHz', 'freqfilter': ['545', '857'], 'specfilter': ['TB', 'EB', 'ET', 'BT', 'BE'], 'nside_desc_map': [1024, 2048]}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
loaded /global/cscratch1/sd/sebibel/smica/SMICAWEIG_K_CMB_SMICA_highell_bins0200_C_l_DX12_smica_4000_6000_0_0_Full.npy
loaded /global/cscratch1/sd/sebibel/weights/DX12/WEIG_K_CMB_0200_C_l_DX12_smica_4000_6000_0_0_Full.npy
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Run with the following settings:
{'Tscale': 'K_CMB', 'mskset': 'lens', 'freqdset': 'NPIPE', 'Spectrum_scale': 'C_l', 'lmax': 4000, 'lmax_mask': 6000, 'freqdatsplit': '', 'smoothing_window': 0, 'max_polynom':

KeyboardInterrupt: 

## with smica propag weights

In [11]:
W_total = np.loadtxt("/global/homes/s/sebibel/ext/smica_propagation/weights_EB_smica_R3.00.txt").reshape(2,7,4001)
cfc = copy.deepcopy(csu.cf)
cfc['pa']['freqdset'] = 'NPIPE'
cfc['pa']['mskset'] = 'lens'
csuc = Config(cfc)
ioc = IO(csuc) 
# full maps
maps = ioc.load_plamap(csuc.cf, field=(0,1,2), nside_out=csu.nside_out)
# maps = trsf_m.process_all(maps)
beamf = ioc.load_beamf(freqcomb=csu.freqcomb)
tmask, pmask, pmask =  io.load_one_mask_forallfreq()

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Run with the following settings:
{'Tscale': 'K_CMB', 'mskset': 'lens', 'freqdset': 'NPIPE', 'Spectrum_scale': 'C_l', 'lmax': 4000, 'lmax_mask': 6000, 'freqdatsplit': '', 'smoothing_window': 0, 'max_polynom': 0, 'num_sim': 5, 'binname': 'SMICA_highell_bins', 'overwrite_cache': True, 'split': '', 'nside_out': None, 'splitvariation': 'GHz', 'freqfilter': ['545', '857'], 'specfilter': ['TB', 'EB', 'ET', 'BT', 'BE'], 'nside_desc_map': [1024, 2048]}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
loaded {'030': '/global/cfs/cdirs/cmb/data/planck2020/npipe/npipe6v20/npipe6v20_030_map.fits', '044': '/global/cfs/cdirs/cmb/data/planck2020/npipe/npipe6v20/npipe6v20_044_map.fits', '070': '/global/cfs/cdirs/cmb/data/planck2020/npipe/npipe6v20/npipe6v20_070_map.fits', '100': '/global/cfs/cdirs/cmb/data/planck2020/npipe/npipe6v20/npipe6v20_100_map.fits', '143': '/global/cfs/cdirs/cmb/data/planck2020/npipe/npipe6v20/npipe6v20_143_map.fits', '217': '/global/cfs/cdirs/c

In [ ]:
for freq in csu.PLANCKMAPFREQ_f:
    print('freq: ', freq)
    ns = csu.nside_out[0] if int(freq) < 100 else csu.nside_out[1]
    alms = pw.map2alm_spin(maps[freq], hp.ud_grade(pmask[freq], nside_out=ns), 2, lmax-1) # full sky QU->EB
    # almT[det] = alms[0]
    almE[freq] = alms[0]
    almB[freq] = alms[1]

nalm = int((lmax)*(lmax-1+2)/2)  
# combalmT = np.zeros((nalm), dtype=np.complex128)
combalmE = np.zeros((nalm), dtype=np.complex128)
combalmB = np.zeros((nalm), dtype=np.complex128)

In [ ]:
for it, det in enumerate(csu.PLANCKMAPFREQ): #weights do not depend on freqfilter, but almE/B do
    if det in csu.PLANCKMAPFREQ_f:
        print('freq: ', det)
        ns = csu.nside_out[0] if int(det) < 100 else csu.nside_out[1]
        # combalmT += hp.almxfl(almT[name], np.squeeze(W[0,m,:]))
        LHFI = "LFI" if int(det)<100 else "HFI"
        if csuc.cf['pa']['freqdset'].startswith('NPIPE'):
            LHFI = "HFI"
        combalmE += hp.almxfl(hp.almxfl(almE[det],1/beamf[str(det)+'-'+str(det)][LHFI][1].data.field(1)[:lmax]), np.squeeze(W_total[0,it,:]*bcorr[it]/(beam5)))
        combalmE = hp.almxfl(combalmE, 1/hp.pixwin(ns, pol=True)[0][:lmax])
#             combalmE = hp.smoothalm(combalmE, fwhm = np.radians(5/60))
        combalmB += hp.almxfl(hp.almxfl(almB[det],1/beamf[str(det)+'-'+str(det)][LHFI][1].data.field(2)[:lmax]), np.squeeze(W_total[1,it,:]*bcorr[it]/(beam5)))
        combalmB = hp.almxfl(combalmB, 1/hp.pixwin(ns, pol=True)[1][:lmax])
#             combalmB = hp.smoothalm(combalmB, fwhm = np.radians(5/60))

propag_CMB = hp.alm2map([np.zeros_like(combalmE), combalmE, combalmB], csu.nside_out[1])
propag_smica_C_lmin_sc = np.array(ps.map2cl_spin(qumap=propag_CMB[1:3], spin=2, mask=pmask['100'], lmax=lmax-1,
    lmax_mask=lmax*2))*1e12 #maps are different scale than processed powerspectra from this' package pipeline, thus *1e12

freq:  030
freq:  044
freq:  070
freq:  100
freq:  143
freq:  217
freq:  353


In [126]:
%matplotlib widget
for freq in range(0,7):
    plt.plot(W_total[0,freq,:]*bcorr[freq]/(beam5), color=colors[freq])
    plt.plot(W_total[1,freq,:]*bcorr[freq]/(beam5), color=colors[freq], ls='--')
plt.ylim((-0.2,1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.2, 1.0)

In [120]:
print(W_total.shape
     )

(2, 7, 4001)


In [71]:
cfc = copy.deepcopy(csu.cf)
cfc['pa']['freqdset'] = 'NPIPE'
cfc['pa']['mskset'] = 'lens'
csuc = Config(cfc)
ioc = IO(csuc)
# CMB = io.load_data(ioc.fh.map_cmb_sc_path_name)
MV = io.load_data(ioc.fh.cmbmap_smica_path_name)

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Run with the following settings:
{'Tscale': 'K_CMB', 'mskset': 'lens', 'freqdset': 'NPIPE', 'Spectrum_scale': 'C_l', 'lmax': 4000, 'lmax_mask': 6000, 'freqdatsplit': '', 'smoothing_window': 0, 'max_polynom': 0, 'num_sim': 5, 'binname': 'SMICA_highell_bins', 'overwrite_cache': True, 'split': '', 'nside_out': None, 'splitvariation': 'GHz', 'freqfilter': ['545', '857'], 'specfilter': ['TB', 'EB', 'ET', 'BT', 'BE'], 'nside_desc_map': [1024, 2048]}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
loaded /global/cscratch1/sd/sebibel/misc/map_cmb_in_nside_2048_sim_id_0200.npy
loaded /global/cscratch1/sd/sebibel/smica/smicaminvarmap_SMICA_highell_bins_0200_C_l_NPIPE_lens_4000_6000_0_0_Full.npy


In [76]:
%matplotlib widget
map_NPDXsmica = hp.alm2map([np.zeros_like(combalmE), combalmE, combalmB], csu.nside_out[1])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [77]:
hp.mollzoom(map_NPDXsmica[2])
hp.mollzoom(MV[2])
hp.mollzoom(map_NPDXsmica[2]-MV[2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …